###############################################################################
# SRSWTIChunkAnalyzer
###############################################################################

In [ ]:
# ---------------------------  Imports -------------------------------
from srswti_axis import SRSWTIChunkAnalyzer
from utils import call_groq_llm, SAMPLE_DOCS
import random


/home/zhreyas/miniconda3/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-10 15:04:31.116608: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741599271.135830  932931 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741599271.141065  932931 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-10 15:04:31.161396: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-c

# --------------------------- Basic Demos -------------------------


In [ ]:
# Initialize the chunk analyzer
chunk_analyzer = SRSWTIChunkAnalyzer()


In [ ]:
# Define a sample sentence to be 
text_c1 = "The quick brown fox jumps over the lazy dog."
print("== Chunk 1 ==")
# Call the analyze_text method of the chunk_analyzer object (an instance of SRSWTIChunkAnalyzer)
# with text_c1 as input, and print the returned analysis to the console
print(chunk_analyzer.analyze_text(text_c1), "\n")

== Chunk 1 ==
{'overall_stats': {'sentence_count': 1, 'total_chunks': 6, 'chunk_distribution': defaultdict(<class 'int'>, {'S': 1, 'CLAUSE': 1, 'NP': 2, 'VP': 1, 'PP': 1})}, 'sentence_analysis': [{'sentence': 'The quick brown fox jumps over the lazy dog.', 'chunks': [{'text': 'The quick brown fox jumps over the lazy dog .', 'type': 'S', 'level': 4, 'start': 0, 'end': 45, 'sub_chunks': [SRSWTIChunk(text='The quick brown fox jumps', type='CLAUSE', level=3, start=0, end=25, sub_chunks=[], grammatical_role=None), SRSWTIChunk(text='over the lazy dog', type='PP', level=3, start=0, end=17, sub_chunks=[], grammatical_role='modifier')], 'grammatical_role': None}, {'text': 'The quick brown fox jumps', 'type': 'CLAUSE', 'level': 3, 'start': 0, 'end': 25, 'sub_chunks': [SRSWTIChunk(text='The quick brown fox', type='NP', level=2, start=0, end=19, sub_chunks=[], grammatical_role=None), SRSWTIChunk(text='jumps', type='VP', level=2, start=0, end=5, sub_chunks=[], grammatical_role='predicate')], 'gramm

In [ ]:
# Define a second sample text containing a cause-and-effect relationship 
# to demonstrate how the chunk analyzer processes financial/business content
text_c2 = "When the CFO left, investors panicked and stock fell."
print("== Chunk 2 ==")
# Process the text using the chunk analyzer and print the resulting analysis
# This will likely identify the causal structure (CFO leaving → investor panic → stock falling)
print(chunk_analyzer.analyze_text(text_c2), "\n")


== Chunk 2 ==
{'overall_stats': {'sentence_count': 1, 'total_chunks': 10, 'chunk_distribution': defaultdict(<class 'int'>, {'S': 1, 'CLAUSE': 3, 'NP': 3, 'VP': 3})}, 'sentence_analysis': [{'sentence': 'When the CFO left, investors panicked and stock fell.', 'chunks': [{'text': 'When the CFO left , investors panicked and stock fell .', 'type': 'S', 'level': 4, 'start': 0, 'end': 55, 'sub_chunks': [SRSWTIChunk(text='the CFO left', type='CLAUSE', level=3, start=0, end=12, sub_chunks=[], grammatical_role=None), SRSWTIChunk(text='investors panicked', type='CLAUSE', level=3, start=0, end=18, sub_chunks=[], grammatical_role=None), SRSWTIChunk(text='stock fell', type='CLAUSE', level=3, start=0, end=10, sub_chunks=[], grammatical_role=None)], 'grammatical_role': None}, {'text': 'the CFO left', 'type': 'CLAUSE', 'level': 3, 'start': 0, 'end': 12, 'sub_chunks': [SRSWTIChunk(text='the CFO', type='NP', level=2, start=0, end=7, sub_chunks=[], grammatical_role=None), SRSWTIChunk(text='left', type='VP

In [ ]:
# Sample three
text_c3 = """
Despite rain, the event was a success.
Attendees praised the resilience of organizers.
"""
print("== Chunk 3 ==")
print(chunk_analyzer.analyze_text(text_c3), "\n")

== Chunk 3 ==
{'overall_stats': {'sentence_count': 2, 'total_chunks': 14, 'chunk_distribution': defaultdict(<class 'int'>, {'S': 2, 'PP': 2, 'NP': 6, 'CLAUSE': 2, 'VP': 2})}, 'sentence_analysis': [{'sentence': '\nDespite rain, the event was a success.', 'chunks': [{'text': 'Despite rain , the event was a success .', 'type': 'S', 'level': 5, 'start': 0, 'end': 40, 'sub_chunks': [SRSWTIChunk(text='Despite rain', type='PP', level=4, start=0, end=12, sub_chunks=[], grammatical_role='modifier'), SRSWTIChunk(text='the event was a success', type='CLAUSE', level=4, start=0, end=23, sub_chunks=[], grammatical_role=None)], 'grammatical_role': None}, {'text': 'Despite rain', 'type': 'PP', 'level': 3, 'start': 0, 'end': 12, 'sub_chunks': [SRSWTIChunk(text='rain', type='NP', level=2, start=0, end=4, sub_chunks=[], grammatical_role=None)], 'grammatical_role': 'modifier'}, {'text': 'rain', 'type': 'NP', 'level': 2, 'start': 0, 'end': 4, 'sub_chunks': [], 'grammatical_role': None}, {'text': 'the event

# -------------------  RAG Demos ----------------------


In [ ]:


def rag_demo_a(docs=SAMPLE_DOCS):
    """
    Demonstration of a basic RAG (Retrieval-Augmented Generation) system.
    
    This function analyzes each document in the provided collection using the chunk analyzer,
    creates a prompt containing the analysis results, and sends it to a language model
    to generate a summary based on the structured document chunks.
    
    Args:
        docs (list): A list of document strings to analyze. Defaults to SAMPLE_DOCS.
        
    Returns:
        None: Results are printed to console.
    """
    chunked_info = [(d, chunk_analyzer.analyze_text(d)) for d in docs]
    print(chunked_info)
    prompt = f"Chunk analyses of docs:\n{chunked_info}\nProvide a summary."
    resp = call_groq_llm(prompt)
    print("[Theorem6 RAG Demo A] LLM:\n", resp, "\n")

def rag_demo_b(user_query, docs=SAMPLE_DOCS):
    """
    Demonstration of a query-based RAG system with chunk-aware responses.
    
    This function simulates document retrieval by randomly selecting a document,
    analyzes it with the chunk analyzer, and then constructs a prompt that
    includes the user query, the selected document, and its chunk structure.
    The LLM is instructed to answer the query while referencing the chunk structure.
    
    Args:
        user_query (str): The question or query from the user.
        docs (list): A list of document strings to choose from. Defaults to SAMPLE_DOCS.
        
    Returns:
        None: Results are printed to console.
    """
    best_doc = random.choice(docs) if docs else "No doc"
    chunks = chunk_analyzer.analyze_text(best_doc).get("chunks", [])
    prompt = (
        f"User query: {user_query}\nBest doc:\n{best_doc}\nChunks:\n{chunks}\n"
        "Answer referencing chunk structure."
    )
    resp = call_groq_llm(prompt)
    print("[Theorem6 RAG Demo B] LLM:\n", resp, "\n")



In [ ]:
rag_demo_a(SAMPLE_DOCS)

[("Neural Networks: An Overview\n\nNeural networks are computational models inspired by the human brain's architecture. They consist of interconnected nodes (neurons) organized in layers: input, hidden, and output layers.\n\nKey Concepts:\n1. Structure: Neurons receive inputs, apply weights, sum them, and pass through activation functions.\n2. Learning: Networks learn by adjusting weights through training processes like backpropagation.\n3. Activation Functions: Functions like ReLU, sigmoid, or tanh determine neuron output.\n\nTypes:\n- Feedforward Neural Networks: Information flows in one direction\n- Convolutional Neural Networks (CNNs): Specialized for image processing\n- Recurrent Neural Networks (RNNs): Handle sequential data with memory\n- Transformers: Advanced architecture for natural language processing\n\nApplications:\n- Image and speech recognition\n- Natural language processing\n- Game playing and decision making\n- Medical diagnosis and predictions\n\nChallenges include o

In [ ]:
rag_demo_b("How does generative AI work?", SAMPLE_DOCS)


[Theorem6 RAG Demo B] LLM:
 [Groq LLM] Generative AI works by utilizing neural networks, which are computational models inspired by the human brain's architecture. These networks consist of layers of interconnected nodes (neurons) that process and transform inputs. 

To understand how generative AI works, let's break down the key concepts related to neural networks, as outlined in the provided text:
1. **Structure**: Neurons in a neural network receive inputs, apply weights, sum them, and pass through activation functions. This process determines the output of each neuron.
2. **Learning**: Neural networks learn through training processes like backpropagation, which adjusts the weights to minimize errors between predicted and actual outputs.
3. **Activation Functions**: Activation functions, such as ReLU, sigmoid, or tanh, determine the output of each neuron.

Generative AI relies on specific types of neural networks:
- **Feedforward Neural Networks**: Information flows in one direction